1- Preprocessing :
define characters accepted and tashkeel accepted and remove from the training set any tashkeel and unwanted characters (eg. brackets, numbers... etc)

In [1]:
import re
import numpy as np
import pyarabic.araby as araby
import nltk
from nltk.tokenize import sent_tokenize
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential,load_model
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from gensim.models import Word2Vec
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
# from tensorflow.keras.callbacks import EarlyStopping

In [8]:
arabicDictionary=['ى', 'ع', 'ظ', 'ح', 'ر', 'س', 'ي', 'ش', 'ض', 'ق', ' ', 'ث', 'ل', 'ص', 'ط', 'ك', 'آ', 'م', 'ا', 'إ', 'ه', 'ز', 'ء', 'أ', 'ف', 'ؤ', 'غ', 'ج', 'ئ', 'د', 'ة', 'خ', 'و', 'ب', 'ذ', 'ت', 'ن']
punctuations = ["،", ":", "؛", "-", "؟"]
#reading the training dataset
f = open(r"train.txt", "r",encoding="utf-8").read()
print(type(f))
print(len(f))
print(f[0])

<class 'str'>
18515271
ق


In [4]:
def cleanDataFromNonArabicLetters(data):
    #regex to keep arabic letters only and remove any other character (eg. brackets, numbers ...etc)
    characters_regex =r'[\s\.\u0600-\u06ff\u0750-\u077f\ufb50-\ufbc1\ufbd3-\ufd3f\ufd50-\ufd8f\ufd50-\ufd8f\ufe70-\ufefc\uFDF0-\uFDFD]+'
    processedData = re.findall(characters_regex,data)
    processedData = " ".join(processedData)
    processedData = re.sub(r"\s+"," " ,processedData) #substitute many spaces with one space only
    return processedData

In [7]:
processedData=cleanDataFromNonArabicLetters(f)
print(len(processedData))
print(processedData[1])

17997403
َ


In [9]:
def removeDiacratics(processedData):
    without_diacritics= araby.strip_diacritics(processedData)
    return without_diacritics

In [11]:
without_diacritics = removeDiacratics(processedData)
print(len(without_diacritics))
print(without_diacritics[1])

10711561
و


In [12]:
#take the procrssed text and separate it into sentences
def generateListsWithDiacraticsandWithout(with_diacratics, without_diacritics):
    #---------------------Preprocessing of words with diacratics------------

    #generate longStringSplited which is the list of procrssed text without brackets and numbers and dots 
    #the rest of the punctuation still there
    longStringWithDecimalPoint=re.sub(r"\n", ' ', with_diacratics) #string of data with diacratics removed endlines from it
    for element in punctuations:
        longStringWithDecimalPoint=re.sub(element, '', longStringWithDecimalPoint) #remove punctuations from the string

    longStringSplited=longStringWithDecimalPoint.split('.') #split the string by dots (segment sentences) and make list of them
    longString=' '.join(longStringSplited)  #long string without decimal points

    #-------------------------------------

    #---------------------Preprocessing of words without diacratics------------
    without_diacratics_longStringWithDecimals=re.sub(r"\n", ' ', without_diacritics) #string of data without diacratics removed endlines from it
    for element in punctuations:
        without_diacratics_longStringWithDecimals=re.sub(element, '', without_diacratics_longStringWithDecimals) #remove punctuations from the string

    longStringSplited_withoutDiacratics=without_diacratics_longStringWithDecimals.split('.') #split the string by dots (segment sentences) and make list of them
    longString_withoutDiacratics=' '.join(longStringSplited_withoutDiacratics)  #long string without decimal points

    #-------------------------------------------------------------------------

    list_of_sentences=[]
    for line in longStringSplited_withoutDiacratics: #list of lists of sentences splitted in words without diacratics (used in embeddings)
        list_of_sentences.append(line.split(" "))
    

    # now the variable called longString has a single string with all the processed words in it 
    listOfwordsWith_Diacritics=list()
    listOfwordsWith_NoDiacritics=list()

    listOfwordsWith_Diacritics=re.sub(r"\s+", ' ', longString)
    listOfwordsWith_Diacritics=listOfwordsWith_Diacritics.split(" ")

    listOfwordsWith_NoDiacritics=re.sub(r"\s+", ' ', longString_withoutDiacratics)
    listOfwordsWith_NoDiacritics=listOfwordsWith_NoDiacritics.split(" ")

    return list_of_sentences,listOfwordsWith_Diacritics,listOfwordsWith_NoDiacritics


In [13]:
list_of_sentences,listOfwordsWith_Diacritics,listOfwordsWith_NoDiacritics= generateListsWithDiacraticsandWithout(processedData, without_diacritics)

In [19]:
print(len(list_of_sentences))
print(list_of_sentences[0])
print(len(listOfwordsWith_NoDiacritics))
print(len(listOfwordsWith_Diacritics))
print(listOfwordsWith_Diacritics[2101000:2101100])
print(listOfwordsWith_NoDiacritics[2101000:2101100])

40836
['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة', 'قوله', 'بلفظ', 'يقتضيه', 'كإنكار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوبه', 'من', 'الدين', 'ضرورة', 'كإلقاء', 'مصحف', 'بقذر', 'وشد', 'زنار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلبس', 'الزنار', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحابه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كالزنديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يستتب', '']
2102054
2102054
['أَوْ', 'سَيِّدُهَا', 'فِي', 'ذَلِكَ', 'جَازَ', 'لِأَنَّ', 'لَهُ', 'غَرَضًا', 'فِي', 'تَزَيُّنِهَا', 'لَهُ', 'كَمَا', 'فِي', 'الرَّوْضَةِ', 'وَهُوَ', 'الْأَوْجَهُ', 'وَإِنْ', 'جَرَى', 'فِي', 'التَّحْقِيقِ', 'عَلَى', 'خِلَافِ', 'ذَلِكَ', 'فِي', 'الْوَصْلِ', 'وَالْوَشْرِ', 'فَأَلْحَقَهُمَا', 'بِالْوَشْمِ', 'فِي', 'الْمَنْعِ', 'مُطْلَقًا', 'وَيُكْرَهُ', 'أَنْ', 'يَنْتِفَ', 'الشَّيْبَ', 'مِنْ', 'الْمَح

In [15]:
def generateDiacraticsList(listOfwordsWith_Diacritics):
    #now that we have two separated lists we need to get the diacritics list 


    listofDiacritrcs_ToWord=list()
    temp=list()
    counter=0
    for word in listOfwordsWith_Diacritics:
        while counter<len(word):
            if word[counter] in arabicDictionary: #checking if the character is a letter
                if (counter+1)<len(word):
                    #checking if the next character is also a letter, then that means that the diacritics of the current letter is none so add empty string to the list
                    if word[counter +1] in arabicDictionary: 
                        temp.append("")
                        counter+=1
                        continue
                counter+=1 #if it is the end of the word (no more letters) or the next character is a diacritics -> continue looping
                continue
            else:
                if (counter+1)<len(word):
                    if word[(counter+1)] not in arabicDictionary: #if the current and the next characters are diacritics, add them together in the list
                        temp.append(word[counter]+word[counter+1])
                        counter+=2
                        continue
                temp.append(word[counter]) #if the current character only is the diacritics add it to the list
                counter+=1    
        listofDiacritrcs_ToWord.append(temp.copy())     
        temp.clear() 
        counter=0
    return listofDiacritrcs_ToWord

In [16]:
listofDiacritrcs_ToWord= generateDiacraticsList(listOfwordsWith_Diacritics)

In [17]:
print(len(listofDiacritrcs_ToWord))
print(listOfwordsWith_Diacritics[3])
print(listOfwordsWith_NoDiacritics[3])
print(listofDiacritrcs_ToWord[3])
print(" ".join(listofDiacritrcs_ToWord[3]))
print(listofDiacritrcs_ToWord[0:100])

2102054
الْأَوَّلُ
الأول
['', 'ْ', 'َ', 'َّ', 'ُ']
 ْ َ َّ ُ
[['َ', 'ْ', 'ُ', 'ُ'], ['َ', 'ْ'], ['َ', 'َ', 'َ'], ['', 'ْ', 'َ', 'َّ', 'ُ'], ['َ', 'َ', 'ُ'], ['', 'َ', 'ْ'], ['َ', '', 'َ'], ['', '', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ', 'ُ'], ['ِ', 'َ', 'ْ', 'ٍ'], ['َ', 'ْ', 'َ', 'ِ', ''], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['َ', 'ْ', 'ِ'], ['َ', 'ِ', '', 'ٍ'], ['ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ُ', '', 'َ'], ['َ'], ['ُ', 'ِ', 'َ'], ['ُ', 'ُ', '', 'ُ', 'ُ'], ['ِ', 'ْ'], ['', '', 'ِّ', '', 'ِ'], ['َ', 'ُ', '', 'َ', 'ً'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['ُ', 'ْ', 'َ', 'ٍ'], ['ِ', 'َ', 'َ', 'ٍ'], ['َ', 'َ', 'ِّ'], ['ُ', 'َّ', '', 'ٍ'], ['', 'ْ', 'ُ'], ['َ', 'َ', 'َ', 'َ'], ['َ', 'ْ', 'ُ'], ['', 'ْ', 'ِ'], ['َ', '', 'ٍ'], ['َ', 'ْ'], ['ِ', 'ِ', 'ْ', 'ٍ'], ['َ', 'َ', 'َ', 'َّ', 'ُ', 'ُ'], ['ُ', 'َ'], ['َ', 'ُ', 'ْ', 'ِ'], ['', '', 'ُّ', 'َّ', '', 'ِ'], ['َ', 'ِ', 'ْ', 'َ', '', 'ِ'], ['', 'ْ', 'ُ', 'ْ', 'َ', 'ِ'], ['ِ'], ['َ', 'ِ', '', 'ِ'],

---

## Trying Bert embeddings

In [18]:
from transformers import  TFBertModel, BertTokenizer, BertModel
import torch

In [31]:
list_of_string_sentences=[]
for i,sentence in enumerate(list_of_sentences):
    list_of_words = list_of_sentences[i]
    sentence_as_string = " ".join(list_of_words)
    list_of_string_sentences.append(sentence_as_string)
print(list_of_string_sentences[0])
print(list_of_string_sentences[1])

قوله أو قطع الأول يده إلخ قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر وشد زنار ابن عرفة قول ابن شاس أو بفعل يتضمنه هو كلبس الزنار وإلقاء المصحف في صريح النجاسة والسجود للصنم ونحو ذلك وسحر محمد قول مالك وأصحابه أن الساحر كافر بالله تعالى قال مالك هو كالزنديق إذا عمل السحر بنفسه قتل ولم يستتب 
 قوله لعدم ما تتعلق إلخ أي الوصية قوله ما مر أي قبيل قول المتن لغت ولو اقتصر على أوصيت له بشاة أو أعطوه شاة ولا غنم له عند الموت هل تبطل الوصية أو يشترى له شاة ويؤخذ من قوله الآتي كما لو لم يقل من مالي ولا من غنمي أنها لا تبطل  وعبارة الكنز ولو لم يقل من مالي ولا من غنمي لم يتعين غنمه إن كانت انتهت ا ه سم قوله فيعطى واحدة منها إلخ كما لو كانت موجودة عند الوصية والموت  ولا يجوز أن يعطى واحدة من غير غنمه في الصورتين وإن تراضيا  لأنه صلح على مجهول مغني ونهاية قال ع ش قوله واحدة منها أي كاملة  ولا يجوز أن يعطى نصفين من شاتين  لأنه لا يسمى شاة وقوله ولا يجوز أن يعطى واحدة من غير غنمه وينبغي أن يقال مثل ذلك في الأرقاء ا ه 


In [36]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#we should get sentence [each sentence is a sequence, should have max of 512 tokens]


In [42]:
tokens_for_all_sentences=[]
max_token_len=0
for sent in list_of_string_sentences:
    tokens_in_sent=tokenizer.tokenize(sent)
    tokens_in_sent = ['[CLS]'] + tokens_in_sent + ['[SEP]']
    # print(tokens_in_sent)
    # print(len(tokens_in_sent))
    max_token_len=max(len(tokens_in_sent),max_token_len)
    tokens_for_all_sentences.append(tokens_in_sent)

print(tokens_for_all_sentences[0])
print(tokens_for_all_sentences[1])
print(max_token_len)

['[CLS]', 'ق', '##و', '##ل', '##ه', 'ا', '##و', 'ق', '##ط', '##ع', 'ا', '##ل', '##ا', '##و', '##ل', 'ي', '##د', '##ه', 'ا', '##ل', '##خ', 'ق', '##ا', '##ل', 'ا', '##ل', '##ز', '##ر', '##ك', '##ش', '##ي', 'ا', '##ب', '##ن', 'ع', '##ر', '##ف', '##ة', 'ق', '##و', '##ل', '##ه', 'ب', '##ل', '##ف', '##ظ', 'ي', '##ق', '##ت', '##ض', '##ي', '##ه', 'ك', '##ان', '##ك', '##ا', '##ر', 'غ', '##ي', '##ر', 'ح', '##د', '##ي', '##ث', 'ب', '##ا', '##ل', '##ا', '##س', '##ل', '##ا', '##م', 'و', '##ج', '##و', '##ب', 'م', '##ا', 'ع', '##ل', '##م', 'و', '##ج', '##و', '##ب', '##ه', 'م', '##ن', 'ا', '##ل', '##د', '##ي', '##ن', 'ض', '##ر', '##و', '##ر', '##ة', 'ك', '##ا', '##ل', '##ق', '##ا', '##ء', 'م', '##ص', '##ح', '##ف', 'ب', '##ق', '##ذ', '##ر', 'و', '##ش', '##د', 'ز', '##ن', '##ا', '##ر', 'ا', '##ب', '##ن', 'ع', '##ر', '##ف', '##ة', 'ق', '##و', '##ل', 'ا', '##ب', '##ن', 'ش', '##ا', '##س', 'ا', '##و', 'ب', '##ف', '##ع', '##ل', 'ي', '##ت', '##ض', '##م', '##ن', '##ه', 'ه', '##و', 'ك', '##ل', '##ب', '##س', 'ا'

In [24]:
tokens = tokens + ['[PAD]'] + ['[PAD]']   # to make a fixed size -- dk yet
print(len(tokens))

294


In [25]:
attention_mask = [1 if i!= '[PAD]' else 0 for i in tokens]
print(attention_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0]


In [26]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[101, 1292, 29836, 23673, 14157, 1270, 29836, 1292, 29828, 29830, 1270, 23673, 25573, 29836, 23673, 1300, 15394, 14157, 1270, 23673, 29821, 1292, 25573, 23673, 1270, 23673, 29823, 17149, 29835, 29825, 14498, 1270, 29816, 15915, 1288, 17149, 29833, 19433, 1292, 29836, 23673, 14157, 1271, 23673, 29833, 29829, 1300, 29834, 29817, 29827, 14498, 14157, 1293, 18511, 29835, 25573, 17149, 1289, 14498, 17149, 1276, 15394, 14498, 29818, 1271, 25573, 23673, 25573, 29824, 23673, 25573, 22192, 1298, 29819, 29836, 29816, 1295, 25573, 1288, 23673, 22192, 1298, 29819, 29836, 29816, 14157, 1295, 15915, 1270, 23673, 15394, 14498, 15915, 1285, 17149, 29836, 17149, 19433, 1293, 25573, 23673, 29834, 25573, 29815, 1295, 29826, 29820, 29833, 1271, 29834, 29822, 17149, 1298, 29825, 15394, 1281, 15915, 25573, 17149, 1270, 29816, 15915, 1288, 17149, 29833, 19433, 1292, 29836, 23673, 1270, 29816, 15915, 1283, 25573, 29824, 1270, 29836, 1271, 29833, 29830, 23673, 1300, 29817, 29827, 22192, 15915, 14157, 1297, 298

In [27]:
token_ids = torch.tensor(token_ids).unsqueeze(0)

attention_mask = torch.tensor(attention_mask).unsqueeze(0)

output = model(token_ids, attention_mask = attention_mask)

In [28]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-2.0725e-01,  2.0815e-01, -3.2501e-01,  ..., -1.9035e-01,
           7.6704e-01, -1.0966e-01],
         [-1.4711e+00,  3.0560e-01,  3.1752e-01,  ...,  4.8062e-02,
           6.0590e-01,  8.6927e-01],
         [-4.6436e-01,  4.7697e-01, -1.0390e-01,  ..., -4.6546e-01,
          -1.1156e-03, -7.7404e-01],
         ...,
         [ 3.8932e-01,  1.0696e-01, -7.2007e-01,  ..., -3.5734e-02,
           3.4067e-02, -4.1209e-01],
         [ 2.0424e-01,  2.5021e-01,  4.6779e-02,  ..., -1.7390e-01,
           7.1936e-01, -4.0982e-01],
         [ 6.9119e-02, -9.3067e-02,  5.2186e-02,  ..., -2.0024e-02,
           5.4998e-01, -1.3584e-01]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-6.9516e-01, -3.4412e-01, -1.8805e-01,  5.5672e-01,  1.8127e-01,
         -1.8612e-01,  2.6372e-01,  2.4826e-01,  2.0408e-01, -9.9983e-01,
          1.1710e-01,  1.6371e-01,  9.5294e-01,  1.1354e-01,  6.3352e-01,
         -2.2570e-

---

## Trying AraBert

In [43]:
from transformers import AutoTokenizer, AutoModel

model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [51]:
max_token_limit = tokenizer.model_max_length
print("Max Token Limit:", max_token_limit)

Max Token Limit: 512


In [52]:
# print(len(list_of_string_sentences))
# print(len(list_of_string_sentences[0]))
# print(list_of_string_sentences[0])
# tokens_for_all_sentences=[]
# max_token_len=0
# min_tokens_len=1000
# for sentence in list_of_string_sentences:
#     tokens_in_sent=tokenizer.tokenize(sentence)
#     tokens_in_sent = ['[CLS]'] + tokens_in_sent + ['[SEP]']
#     max_token_len=max(len(tokens_in_sent),max_token_len)
#     min_tokens_len=min(len(tokens_in_sent), min_tokens_len)
#     tokens_for_all_sentences.append(tokens_in_sent)

# print(len(tokens_for_all_sentences))
# print(len(tokens_for_all_sentences[0]))
# print(tokens_for_all_sentences[0])
# print(max_token_len)
# print(min_tokens_len)

40836
361
قوله أو قطع الأول يده إلخ قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر وشد زنار ابن عرفة قول ابن شاس أو بفعل يتضمنه هو كلبس الزنار وإلقاء المصحف في صريح النجاسة والسجود للصنم ونحو ذلك وسحر محمد قول مالك وأصحابه أن الساحر كافر بالله تعالى قال مالك هو كالزنديق إذا عمل السحر بنفسه قتل ولم يستتب 
40836
93
['[CLS]', 'قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزر', '##كش', '##ي', 'ابن', 'عرفة', 'قوله', 'بلف', '##ظ', 'يقتضي', '##ه', 'كإ', '##نك', '##ار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوب', '##ه', 'من', 'الدين', 'ضرورة', 'كإ', '##لقاء', 'مصح', '##ف', 'بق', '##ذر', 'وشد', 'زن', '##ار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلب', '##س', 'الزنا', '##ر', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجا', '##سة', 'والس', '##جو', '##د', 'للص', '##نم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحاب', '##ه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو

In [59]:
# max_token_limit = 512  # or any other desired limit
# stride = 256  # or any other desired overlap

# tokens_for_all_sentences = []

# for sentence in list_of_string_sentences:
#     tokens_in_sent = tokenizer.tokenize(sentence)
#     segments = []

#     # Create overlapping segments
#     for i in range(0, len(tokens_in_sent), stride):
#         segment = ['[CLS]'] + tokens_in_sent[i:i + stride - 2] + ['[SEP]']
#         segments.append(segment)

#     tokens_for_all_sentences.extend(segments)


# print(len(tokens_for_all_sentences))
# print(len(tokens_for_all_sentences[0]))
# print(tokens_for_all_sentences[0])



41494
93
['[CLS]', 'قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزر', '##كش', '##ي', 'ابن', 'عرفة', 'قوله', 'بلف', '##ظ', 'يقتضي', '##ه', 'كإ', '##نك', '##ار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوب', '##ه', 'من', 'الدين', 'ضرورة', 'كإ', '##لقاء', 'مصح', '##ف', 'بق', '##ذر', 'وشد', 'زن', '##ار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلب', '##س', 'الزنا', '##ر', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجا', '##سة', 'والس', '##جو', '##د', 'للص', '##نم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحاب', '##ه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كال', '##زن', '##ديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يست', '##تب', '[SEP]']


In [56]:
# constant_token_len = 256  # Set the desired maximum token length

# tokens_for_all_sentences = []

# for sentence in list_of_string_sentences:
#     tokens_in_sent = tokenizer.tokenize(sentence)
    
#     # Truncate or pad to the desired length
#     tokens_in_sent = ['[CLS]'] + tokens_in_sent[:constant_token_len - 2] + ['[SEP]']
#     padding_length = max_token_len - len(tokens_in_sent)
#     tokens_in_sent += ['[PAD]'] * padding_length
    
#     tokens_for_all_sentences.append(tokens_in_sent)

# print(len(tokens_for_all_sentences))
# print(len(tokens_for_all_sentences[0]))
# print(tokens_for_all_sentences[0])



40836
256
['[CLS]', 'قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزر', '##كش', '##ي', 'ابن', 'عرفة', 'قوله', 'بلف', '##ظ', 'يقتضي', '##ه', 'كإ', '##نك', '##ار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوب', '##ه', 'من', 'الدين', 'ضرورة', 'كإ', '##لقاء', 'مصح', '##ف', 'بق', '##ذر', 'وشد', 'زن', '##ار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلب', '##س', 'الزنا', '##ر', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجا', '##سة', 'والس', '##جو', '##د', 'للص', '##نم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحاب', '##ه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كال', '##زن', '##ديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يست', '##تب', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [95]:
max_token_len = 256  # Set the desired maximum token length

tokens_for_all_sentences = []

for sentence in list_of_string_sentences:
    tokens_in_sent = tokenizer.tokenize(sentence)
    
    # If the sequence is shorter than 256 tokens, pad it
    if len(tokens_in_sent) < max_token_len - 2:
        tokens_in_sent=['[CLS]'] + tokens_in_sent + ['[SEP]']
        padding_length = max_token_len - len(tokens_in_sent)
        tokens_in_sent += ['[PAD]'] * padding_length
        tokens_for_all_sentences.append(tokens_in_sent)

    # If the sequence is longer than 256 tokens, segment it
    else:
        segments = []
        # Create overlapping segments
        for i in range(0, len(tokens_in_sent), max_token_len-2):
            segment = ['[CLS]'] + tokens_in_sent[i:i + max_token_len - 2] + ['[SEP]']
            #Now we need to check that segment reach 256
            if(len(segment)<256):
                padding_length = max_token_len - len(segment)
                segment += ['[PAD]'] * padding_length

            segments.append(segment)
        tokens_for_all_sentences.extend(segments)

print(len(tokens_for_all_sentences))
print(len(tokens_for_all_sentences[0]))
print(tokens_for_all_sentences[0])
print(len(tokens_for_all_sentences[42113]))
print(tokens_for_all_sentences[42113])
print(len(tokens_for_all_sentences[42114]))
print(tokens_for_all_sentences[42114])



42115
256
['[CLS]', 'قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزر', '##كش', '##ي', 'ابن', 'عرفة', 'قوله', 'بلف', '##ظ', 'يقتضي', '##ه', 'كإ', '##نك', '##ار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوب', '##ه', 'من', 'الدين', 'ضرورة', 'كإ', '##لقاء', 'مصح', '##ف', 'بق', '##ذر', 'وشد', 'زن', '##ار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلب', '##س', 'الزنا', '##ر', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجا', '##سة', 'والس', '##جو', '##د', 'للص', '##نم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحاب', '##ه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كال', '##زن', '##ديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يست', '##تب', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PA

In [96]:
# Check lengths of sequences
sum=0
for i, seq in enumerate(tokens_for_all_sentences):
    if(len(seq)!=256):
        sum=sum+1
        print(f"Sequence {i+1} length: {len(seq)}")
print(sum)


0


In [97]:
token_ids_for_sentences=[]
for tokens_in_sentence in tokens_for_all_sentences:
    token_ids_for_sentences.append(tokenizer.convert_tokens_to_ids(tokens_in_sentence))
print(token_ids_for_sentences[0])
print(token_ids_for_sentences[42114])
#I am not sure if CLS and SEP have token IDs of 2 and 3 or 102 and 103?

[2, 4151, 440, 4377, 810, 11035, 22323, 754, 19567, 19593, 193, 2298, 25425, 4151, 33729, 231, 24607, 195, 15527, 12978, 304, 650, 2643, 30771, 14513, 394, 1805, 14513, 195, 306, 1220, 2096, 15527, 10648, 24950, 198, 1211, 1871, 41872, 26823, 304, 2298, 25425, 5316, 2298, 18644, 440, 8842, 51492, 583, 30897, 213, 43496, 183, 29583, 28919, 305, 22300, 12007, 438, 1573, 6000, 196, 3373, 13361, 19042, 563, 56110, 582, 5316, 8210, 11674, 195, 331, 33948, 59474, 10515, 5698, 754, 8210, 583, 2818, 2126, 4299, 1217, 733, 22413, 8970, 2222, 1174, 885, 13655, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [98]:
# Check lengths of sequences
sum=0
for i, seq in enumerate(token_ids_for_sentences):
    if(len(seq)!=256):
        sum=sum+1
        print(f"Sequence {i+1} length: {len(seq)}")
print(sum)


0


In [68]:
list(tokenizer.vocab.keys())[5000:5020]

['[UNUSED_952]',
 'هتاف',
 'متى',
 'جلسته',
 'وأنشأ',
 '[UNUSED_2168]',
 '##هاء',
 'ادارات',
 'المشغولات',
 '##ابرز',
 '##واري',
 'يحترم',
 'جاهدا',
 'مساره',
 '192',
 'بوسائل',
 '##فاضلة',
 'واستخراج',
 'المبرمة',
 'والحد']

In [69]:
tokens_to_check = ["[CLS]", "[SEP]"]

# Convert tokens to token IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens_to_check)

print(token_ids)


[2, 3]


In [99]:
# Mark each of the tokens as belonging to sentence "1".
segment_ids_in_sentences=[]
for i,segment_sent in enumerate(token_ids_for_sentences):
    segment_ids= [i]* len(segment_sent)
    segment_ids_in_sentences.append(segment_ids)


In [100]:
print(len(segment_ids_in_sentences))
print(segment_ids_in_sentences[0])
print(segment_ids_in_sentences[42114])

42115
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 42114, 

In [101]:
print(len(token_ids_for_sentences))
print(len(segment_ids_in_sentences))

print(len(token_ids_for_sentences[800]))


42115
42115
256


In [112]:
# Convert token IDs and segment IDs to PyTorch tensors
tokens_tensor = torch.tensor(token_ids_for_sentences)
segments_tensor = torch.tensor(segment_ids_in_sentences)

batch_size = 100  # Set an appropriate batch size
tokens_tensor = tokens_tensor[:batch_size]
segments_tensor = segments_tensor[:batch_size]



In [114]:

# Forward pass through the model to get embeddings
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensor)

# Extract embeddings from the last hidden states
last_hidden_states = outputs.last_hidden_state

# Now, 'last_hidden_states' contains the embeddings for each token in each sentence
print(last_hidden_states.shape)   # (batch size, sequence length, hidden size)  # all sequences must have 256 length
print(last_hidden_states[0,:,:])

torch.Size([100, 256, 768])
tensor([[ 0.7184,  1.4945,  0.5356,  ..., -0.8636, -0.2222, -0.0933],
        [ 0.6897,  1.5559,  0.5676,  ..., -0.8897, -0.3539, -0.1131],
        [ 0.6841,  1.6203,  0.5395,  ..., -0.8518, -0.3706, -0.0906],
        ...,
        [ 0.6723,  1.3007,  0.5677,  ..., -0.8933, -0.2771, -0.0411],
        [ 0.6737,  1.3088,  0.5674,  ..., -0.8945, -0.2773, -0.0404],
        [ 0.6749,  1.3187,  0.5686,  ..., -0.8955, -0.2789, -0.0407]])


In [ ]:
# Note that we can do pooling for sentence-level embeddings (batch size, hidden size)

In [ ]:
# tokens_tensor = torch.tensor([indexed_tokens])
# segments_tensors = torch.tensor([segments_ids])

In [ ]:
# outputs= model(tokens_tensor,segments_tensors)

---

2-Word Embeddings

In [62]:
def word2vecmodel(input):
    # Train Word2Vec model
    Word2Vec_model = Word2Vec(input, vector_size=100, window=5, min_count=1, workers=4, sg=0)
    Word2Vec_model.train(input,total_examples=len(listOfwordsWith_NoDiacritics),epochs=10)
    word_embeddings = {word: Word2Vec_model.wv[word] for word in Word2Vec_model.wv.index_to_key}
    return Word2Vec_model, word_embeddings

In [67]:
word_embeddings_model, word_embeddings= word2vecmodel(list_of_sentences)

3-Building the LSTM model for character level classifier

In [68]:
def formatingXandYtrain(listofDiacritrcs_ToWord,listOfwordsWith_NoDiacritics):
    for i in range(len(listofDiacritrcs_ToWord)):
        listofDiacritrcs_ToWord[i] = " ".join(listofDiacritrcs_ToWord[i])
    Y_train= np.array([listofDiacritrcs_ToWord],dtype=object).T
    X_train = np.array([listOfwordsWith_NoDiacritics],dtype=object)
    print(str(X_train.shape))
    print(str(Y_train.shape))
    return X_train,Y_train
    

In [65]:
X_train,Y_train= formatingXandYtrain(listofDiacritrcs_ToWord,listOfwordsWith_NoDiacritics)

(1, 2102054)
(2102054, 1)


In [75]:
def trainDiacritics(listOfwordsWith_NoDiacritics,listofDiacritrcs_ToWord):
    # Tokenize the input words and diacritics
    word_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
    word_tokenizer.fit_on_texts(listOfwordsWith_NoDiacritics)
    word_sequences = word_tokenizer.texts_to_sequences(listOfwordsWith_NoDiacritics)

    diacritic_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters='')
    diacritic_tokenizer.fit_on_texts([''.join(d) for d in listofDiacritrcs_ToWord])
    diacritic_sequences = diacritic_tokenizer.texts_to_sequences([''.join(d) for d in listofDiacritrcs_ToWord])

    # Pad sequences to have the same length
    max_len = max(max(len(seq) for seq in word_sequences), max(len(seq) for seq in diacritic_sequences))
    padded_word_sequences = pad_sequences(word_sequences, maxlen=max_len, padding='post')
    padded_diacritic_sequences = pad_sequences(diacritic_sequences, maxlen=max_len, padding='post')

    # Build the LSTM model
    model = Sequential()
    model.add(Embedding(input_dim=len(word_tokenizer.word_index) + 1, output_dim=50, input_length=max_len))
    model.add(LSTM(100, return_sequences=True))
    model.add(Dense(len(diacritic_tokenizer.word_index) + 1, activation='softmax'))

    # Compile the model
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Train the model
    model.fit(padded_word_sequences, np.expand_dims(padded_diacritic_sequences, -1), epochs=10, batch_size=32)

    # Save the model for later use
    model.save("diacritic_prediction_model.h5")

    return model,word_tokenizer,diacritic_tokenizer,max_len

# Now, you can use the trained model to predict diacritics for new Arabic words
def predict_diacritics(model,word_tokenizer, diacritic_tokenizer,max_len, word):
    word_sequence = word_tokenizer.texts_to_sequences([word])
    padded_word_sequence = pad_sequences(word_sequence, maxlen=max_len, padding='post')
    predicted_diacritic_sequence = model.predict(padded_word_sequence)
    predicted_diacritic_sequence = np.argmax(predicted_diacritic_sequence, axis=-1)
    predicted_diacritic = diacritic_tokenizer.sequences_to_texts(predicted_diacritic_sequence)
    return predicted_diacritic[0]




In [76]:
model,word_tokenizer,diacritic_tokenizer,max_len=trainDiacritics(listOfwordsWith_NoDiacritics,listofDiacritrcs_ToWord)

Epoch 1/10


In [72]:
model = load_model('diacritic_prediction_model.h5')

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.

In [26]:
# Example usage
# input_word = "الذي علم بالقلم"
# input_word = "ذهب علي إلى الشاطئ"
input = "قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر"
input_word = input.split(' ')
word_vectors = [word_embeddings_model.wv[word] for word in input_word if word in Word2Vec_model.wv]
# print(word_vectors)

for word in input_word: 
  predicted_diacritics1=predict_diacritics(model,word_tokenizer, diacritic_tokenizer,max_len, word)
  print(list(predicted_diacritics1))
  print(f"Input word: {word}")
  print(f"Predicted diacritics: {predicted_diacritics1}")
  tempString=str()
  counterWord=0
  counterDiacrtic=0
  while counterDiacrtic<(len(predicted_diacritics1)):
    if predicted_diacritics1[counterDiacrtic]!=' ':
      tempString=tempString+word[counterWord]
      tempString=tempString+predicted_diacritics1[counterDiacrtic]
      print(predicted_diacritics1[counterDiacrtic])
      if predicted_diacritics1[counterDiacrtic] == 'ّ':
        print("here")
        tempString=tempString+predicted_diacritics1[counterDiacrtic+2]
        counterDiacrtic=counterDiacrtic+2
      counterWord+=1
    counterDiacrtic+=1

  print(tempString)

1/1 [==============================] - 0s 19ms/step
['َ', ' ', ' ', ' ', ' ', ' ', 'َ']
Input word: قال
Predicted diacritics: َ     َ
َ
َ
قَاَ
1/1 [==============================] - 0s 19ms/step
[' ', ' ', 'ْ', ' ', 'ّ', ' ', 'َ', ' ', ' ', ' ', 'ْ', ' ', ' ', ' ', 'َ', ' ', ' ', ' ', 'ِ', ' ', ' ', ' ', 'ّ', ' ', 'ُ']
Input word: الزركشي
Predicted diacritics:   ْ ّ َ   ْ   َ   ِ   ّ ُ
ْ
ّ
here
ْ
َ
ِ
ّ
here
اْلَّزْرَكِشُّ
1/1 [==============================] - 0s 17ms/step
[' ', ' ', 'ْ', ' ', ' ', ' ', 'ِ']
Input word: ابن
Predicted diacritics:   ْ   ِ
ْ
ِ
اْبِ
1/1 [==============================] - 0s 17ms/step
['َ', ' ', ' ', ' ', 'َ', ' ', ' ', ' ', 'َ', ' ', ' ', ' ', 'َ']
Input word: عرفة
Predicted diacritics: َ   َ   َ   َ
َ
َ
َ
َ
عَرَفَةَ
1/1 [==============================] - 0s 17ms/step
['َ', ' ', ' ', ' ', 'ْ', ' ', ' ', ' ', 'ُ', ' ', ' ', ' ', 'ُ']
Input word: قوله
Predicted diacritics: َ   ْ   ُ   ُ
َ
ْ
ُ
ُ
قَوْلُهُ
1/1 [==============================] - 0s 17ms/step
['ِ